In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#import csv and display first 5 rows
survivors = pd.read_csv('train.csv')
survivors.head(5)

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [3]:
#Use describe() to get a quick glance at the structure of the data. 
#NOTE: Columns with text values are omitted
survivors.describe()

PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200

In [4]:
#Alter the data to be more ML friendly

#THis is a list of bins for "Age". ML does better with discrete data rather than continuou. 
#This turns ages into age brackets
bins= (0,5,10,15,20,25,30,35,40,45,50,55,60,65, 70, 75,80,85) 

#Turn Text data into categorical data. Ie. "Male" becomes 1 and "Female" becomes 2.  
survivors["Sex"]=survivors["Sex"].astype("category").cat.codes
survivors["Embarked"]=survivors["Embarked"].astype("category").cat.codes
survivors["Embarked"]=survivors["Embarked"].astype("category").cat.codes

#ML cannot handle Null values or Na values. 
#This fills any NaN in Column "Age" with the average age. 
#Idk if this is appropriate but I read somewhere it's ok.  
#The purpose is to get as much data as possible. 
survivors["Age"]=survivors["Age"].fillna(29.699118)

In [5]:
#This step drops unused columns from the dataframe
survivors.drop(["Name", "Ticket", "Cabin"], axis=1, inplace=True)

In [6]:
#checks the number of NaN's in column age. There should be 0 since they were filled with 29.xxx.
survivors["Age"].isna().sum()

0

In [7]:
# Assign the data to X and y

#Drop any remaining Na values. ML cannot handle them. 
survivors.dropna(inplace=True)
#Assign X and y values
#X is our input and y is our output
X = survivors[["Pclass","Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
y = survivors["Survived"].values.reshape(-1, 1)

#X and y must have same 1st dimension (891)
print(X.shape, y.shape)

(891, 7) (891, 1)


In [8]:
#split the data into training and testing sets
#random state allows us to randomly select data but be able to reuse that same random data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

668
668
223
223


In [9]:
#import the ML model we want to use
from sklearn.linear_model import LogisticRegression
#Instantiate the class LogisticRegression()
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [10]:
#train our chosen model using our data
classifier.fit(X_train, y_train)

C:\Users\warne\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\warne\Anaconda3\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [11]:
#Print our scores
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.7994011976047904
Testing Data Score: 0.8026905829596412


In [14]:
#get a list of our predictions
predictions = classifier.predict(X_test)

#create a more useful report on our model
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions,target_names=["Neg","Pos"]))#Note: I'm not sure if Neg/Pos is correct. Need to verify.

              precision    recall  f1-score   support

         Neg       0.81      0.88      0.85       137
         Pos       0.78      0.67      0.72        86

   micro avg       0.80      0.80      0.80       223
   macro avg       0.80      0.78      0.79       223
weighted avg       0.80      0.80      0.80       223



In [ ]:
# predictions = classifier.predict(X_test)
# pd.DataFrame({"Prediction": predictions, "Actual": np.ravel(y_test)})